In [1]:
import pandas as pd
import pyspark
sale_train=pd.read_csv('https://willtesthdistorage.blob.core.windows.net/data/sales_train.csv')
sale_train.head()

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,Current session?
2,application_1586403990396_0006,pyspark,idle,Link,Link,✔


SparkSession available as 'spark'.
         date  date_block_num  shop_id  item_id  item_price  item_cnt_day
0  02.01.2013               0       59    22154      999.00           1.0
1  03.01.2013               0       25     2552      899.00           1.0
2  05.01.2013               0       25     2552      899.00          -1.0
3  06.01.2013               0       25     2554     1709.05           1.0
4  15.01.2013               0       25     2555     1099.00           1.0

In [2]:
sale_train= spark.createDataFrame(sale_train)
sale_train.show()

+----------+--------------+-------+-------+----------+------------+
|      date|date_block_num|shop_id|item_id|item_price|item_cnt_day|
+----------+--------------+-------+-------+----------+------------+
|02.01.2013|             0|     59|  22154|     999.0|         1.0|
|03.01.2013|             0|     25|   2552|     899.0|         1.0|
|05.01.2013|             0|     25|   2552|     899.0|        -1.0|
|06.01.2013|             0|     25|   2554|   1709.05|         1.0|
|15.01.2013|             0|     25|   2555|    1099.0|         1.0|
|10.01.2013|             0|     25|   2564|     349.0|         1.0|
|02.01.2013|             0|     25|   2565|     549.0|         1.0|
|04.01.2013|             0|     25|   2572|     239.0|         1.0|
|11.01.2013|             0|     25|   2572|     299.0|         1.0|
|03.01.2013|             0|     25|   2573|     299.0|         3.0|
|03.01.2013|             0|     25|   2574|     399.0|         2.0|
|05.01.2013|             0|     25|   2574|     

In [3]:
item_categories='https://willtesthdistorage.blob.core.windows.net/data/item_categories.csv'
item_categories=pd.read_csv(item_categories)
item_categories.head(25)
item_categories= spark.createDataFrame(item_categories)
item_categories.show()

+--------------------+----------------+
|  item_category_name|item_category_id|
+--------------------+----------------+
|PC - ÐÐ°ÑÐ½Ð¸Ñ...|               0|
|ÐÐºÑÐµÑÑÑÐ°Ñ...|               1|
|ÐÐºÑÐµÑÑÑÐ°Ñ...|               2|
|ÐÐºÑÐµÑÑÑÐ°Ñ...|               3|
|ÐÐºÑÐµÑÑÑÐ°Ñ...|               4|
|ÐÐºÑÐµÑÑÑÐ°Ñ...|               5|
|ÐÐºÑÐµÑÑÑÐ°Ñ...|               6|
|ÐÐºÑÐµÑÑÑÐ°Ñ...|               7|
|ÐÐ¸Ð»ÐµÑÑ (Ð¦Ð...|               8|
|ÐÐ¾ÑÑÐ°Ð²ÐºÐ° ...|               9|
|ÐÐ³ÑÐ¾Ð²ÑÐµ Ðº...|              10|
|ÐÐ³ÑÐ¾Ð²ÑÐµ Ðº...|              11|
|ÐÐ³ÑÐ¾Ð²ÑÐµ Ðº...|              12|
|ÐÐ³ÑÐ¾Ð²ÑÐµ Ðº...|              13|
|ÐÐ³ÑÐ¾Ð²ÑÐµ Ðº...|              14|
|ÐÐ³ÑÐ¾Ð²ÑÐµ Ðº...|              15|
|ÐÐ³ÑÐ¾Ð²ÑÐµ Ðº...|              16|
|ÐÐ³ÑÐ¾Ð²ÑÐµ Ðº...|              17|
|      ÐÐ³ÑÑ - PS2|              18|
|      ÐÐ³ÑÑ - PS3|              19|
+--------------------+----------------+
only showing top 20 rows

In [4]:
item='https://willtesthdistorage.blob.core.windows.net/data/items.csv'
item=pd.read_csv(item)
item=spark.createDataFrame(item)
shop='https://willtesthdistorage.blob.core.windows.net/data/shops.csv'
shop=pd.read_csv(shop)
shop=spark.createDataFrame(shop)
test='https://willtesthdistorage.blob.core.windows.net/data/test.csv'
test=pd.read_csv(test)
sale_test=spark.createDataFrame(test)

In [5]:
sale_test.show()

+---+-------+-------+
| ID|shop_id|item_id|
+---+-------+-------+
|  0|      5|   5037|
|  1|      5|   5320|
|  2|      5|   5233|
|  3|      5|   5232|
|  4|      5|   5268|
|  5|      5|   5039|
|  6|      5|   5041|
|  7|      5|   5046|
|  8|      5|   5319|
|  9|      5|   5003|
| 10|      5|   4806|
| 11|      5|   4843|
| 12|      5|   4607|
| 13|      5|   4869|
| 14|      5|   4870|
| 15|      5|   4872|
| 16|      5|   4874|
| 17|      5|   4678|
| 18|      5|   4892|
| 19|      5|   4964|
+---+-------+-------+
only showing top 20 rows

#  Data Cleanup And Table Joining To Create Single Feature Vectors

In [6]:
from pyspark.sql import types,functions as F
from pyspark.sql.functions import unix_timestamp

In [7]:
def cast_types(df):
    dtypes = df.dtypes
    if "ID" in df.columns:
        df = df.withColumn("ID", df["ID"].cast(types.IntegerType()))
    if "date" in df.columns:
        #df = df.withColumn("date",df['date'].cast(types.DateType()))
        df = df.withColumn("date", F.from_unixtime(unix_timestamp('date', 'dd.MM.yyyy')))
    if "date_block_num" in df.columns:
        df = df.withColumn("date_block_num", df["date_block_num"].cast(types.ByteType()))
    if "shop_id" in df.columns:
        df = df.withColumn("shop_id", df["shop_id"].cast(types.ByteType()))
    if "item_id" in df.columns:
        df = df.withColumn("item_id", df["item_id"].cast(types.ShortType()))
    if "item_price" in df.columns:
        df = df.withColumn("item_price", df["item_price"].cast(types.FloatType()))
    if "item_cnt_day" in df.columns:
        df = df.withColumn("item_cnt_day", df["item_cnt_day"].cast(types.FloatType()))
    return df

In [8]:
sale_train=cast_types(sale_train)

In [9]:
train = sale_train.join(item, on='item_id').join(shop, on='shop_id').join(item_categories, on='item_category_id')
print(train.count())
train.show()

2935849
+----------------+-------+-------+-------------------+--------------+----------+------------+--------------------+--------------------+--------------------+
|item_category_id|shop_id|item_id|               date|date_block_num|item_price|item_cnt_day|           item_name|           shop_name|  item_category_name|
+----------------+-------+-------+-------------------+--------------+----------+------------+--------------------+--------------------+--------------------+
|              26|     55|   5230|2014-12-21 00:00:00|            23|     135.2|         2.0|Need for Speed Mo...|Ð¦Ð¸ÑÑÐ¾Ð²Ð¾Ð¹ ...|ÐÐ³ÑÑ Android ...|
|              26|     55|   5230|2015-02-10 00:00:00|            25|     135.2|         1.0|Need for Speed Mo...|Ð¦Ð¸ÑÑÐ¾Ð²Ð¾Ð¹ ...|ÐÐ³ÑÑ Android ...|
|              26|     55|   5230|2015-02-13 00:00:00|            25|     135.2|         1.0|Need for Speed Mo...|Ð¦Ð¸ÑÑÐ¾Ð²Ð¾Ð¹ ...|ÐÐ³ÑÑ Android ...|
|              26|     55|   5230|2015-02-18 00:00

In [10]:
test_shop_ids =sale_test.select('shop_id').distinct()
test_item_ids = sale_test.select('item_id').distinct()
train_filtered = train.join(test_shop_ids, "shop_id")
train_filtered = train_filtered.join(test_item_ids, "item_id")

In [11]:
print('Filtered Training Set Size: %s' % train_filtered.count())
train_filtered.show()

Filtered Training Set Size: 1224439
+-------+-------+----------------+-------------------+--------------+----------+------------+--------------------+--------------------+--------------------+
|item_id|shop_id|item_category_id|               date|date_block_num|item_price|item_cnt_day|           item_name|           shop_name|  item_category_name|
+-------+-------+----------------+-------------------+--------------+----------+------------+--------------------+--------------------+--------------------+
|    474|     22|              49|2013-03-13 00:00:00|             2|     500.0|         1.0|1Ð¡:ÐÑÑÐ³Ð°Ð»Ñ...|ÐÐ¾ÑÐºÐ²Ð° ÐÐ°...|ÐÐ½Ð¸Ð³Ð¸ - ÐÐµ...|
|    474|     22|              49|2013-03-12 00:00:00|             2|     500.0|         3.0|1Ð¡:ÐÑÑÐ³Ð°Ð»Ñ...|ÐÐ¾ÑÐºÐ²Ð° ÐÐ°...|ÐÐ½Ð¸Ð³Ð¸ - ÐÐµ...|
|    474|     22|              49|2013-04-18 00:00:00|             3|     500.0|         1.0|1Ð¡:ÐÑÑÐ³Ð°Ð»Ñ...|ÐÐ¾ÑÐºÐ²Ð° ÐÐ°...|ÐÐ½Ð¸Ð³Ð¸ - ÐÐµ...|
|    474|     22|     

In [12]:
train_monthly = train_filtered[['date', 'date_block_num', 'shop_id', 'item_category_id', 'item_id', 'item_price', 'item_cnt_day']]
train_monthly.show()

+-------------------+--------------+-------+----------------+-------+----------+------------+
|               date|date_block_num|shop_id|item_category_id|item_id|item_price|item_cnt_day|
+-------------------+--------------+-------+----------------+-------+----------+------------+
|2013-02-14 00:00:00|             1|     26|              49|    474|     500.0|         1.0|
|2013-04-30 00:00:00|             3|     26|              49|    474|     500.0|         1.0|
|2014-02-15 00:00:00|            13|     26|              49|    474|     500.0|         1.0|
|2014-10-15 00:00:00|            21|     25|              49|    474|     500.0|         1.0|
|2015-08-18 00:00:00|            31|     25|              49|    474|     500.0|         1.0|
|2013-09-14 00:00:00|             8|     41|              49|    474|     500.0|         1.0|
|2014-06-16 00:00:00|            17|     12|              49|    474|     500.0|         1.0|
|2014-07-28 00:00:00|            18|     12|              49

In [13]:
from pyspark.sql.types import * 
shop_ids=train_monthly.select('shop_id').distinct().collect()

In [14]:
item_ids=train_monthly.select('item_id').distinct().collect()
item_ids

[Row(item_id=4900), Row(item_id=9465), Row(item_id=1238), Row(item_id=1829), Row(item_id=6620), Row(item_id=11141), Row(item_id=10206), Row(item_id=18498), Row(item_id=18944), Row(item_id=4935), Row(item_id=17420), Row(item_id=19204), Row(item_id=496), Row(item_id=18911), Row(item_id=18654), Row(item_id=833), Row(item_id=13289), Row(item_id=7982), Row(item_id=2866), Row(item_id=20735), Row(item_id=5156), Row(item_id=15447), Row(item_id=13285), Row(item_id=1483), Row(item_id=2235), Row(item_id=6393), Row(item_id=14423), Row(item_id=18479), Row(item_id=1721), Row(item_id=12715), Row(item_id=18595), Row(item_id=14075), Row(item_id=18201), Row(item_id=18467), Row(item_id=13261), Row(item_id=22004), Row(item_id=8803), Row(item_id=10230), Row(item_id=4190), Row(item_id=10914), Row(item_id=20396), Row(item_id=20928), Row(item_id=858), Row(item_id=11280), Row(item_id=19325), Row(item_id=11710), Row(item_id=16534), Row(item_id=9162), Row(item_id=7879), Row(item_id=10862), Row(item_id=4391), Row

In [15]:
shop_item_pairs=[]
for i in range(34):
    for shop in shop_ids:
        for item in item_ids:
            shop_item_pairs.append([int(i), int(shop.__getitem__('shop_id')), int(item.__getitem__('item_id'))])

In [16]:
shop_item_pairs=spark.createDataFrame(shop_item_pairs,['date_block_num','shop_id', 'item_id'])

In [17]:
train_monthly=shop_item_pairs.join(train_monthly, on=['date_block_num','shop_id', 'item_id'],how='left')

In [18]:
train_monthly=train_monthly.fillna(0)

In [19]:
train_monthly.describe().show()

+-------+------------------+------------------+------------------+-------------------+------------------+------------------+------------------+
|summary|    date_block_num|           shop_id|           item_id|               date|  item_category_id|        item_price|      item_cnt_day|
+-------+------------------+------------------+------------------+-------------------+------------------+------------------+------------------+
|  count|           7358728|           7358728|           7358728|            1224439|           7358728|           7358728|           7358728|
|   mean|16.676605793827413| 31.69327987119513|10888.203844061094|               null| 6.748686185982143|171.49535016223365| 0.219843837141419|
| stddev|   9.7630333072685|17.439258364680217| 6245.487926484355|               null|16.905063141312166| 838.4413834392589|1.4382110317988603|
|    min|                 0|                 2|                30|2013-01-01 00:00:00|                 0|               0.0|            

In [20]:
year = F.udf(lambda x: ((x//12) + 2013), IntegerType())
month = F.udf(lambda x: (x % 12), IntegerType())

In [21]:
train_monthly = train_monthly.withColumn("year", year(train_monthly.date_block_num))
train_monthly = train_monthly.withColumn("month", month(train_monthly.date_block_num))
train_monthly.show(10)

+--------------+-------+-------+-------------------+----------------+----------+------------+----+-----+
|date_block_num|shop_id|item_id|               date|item_category_id|item_price|item_cnt_day|year|month|
+--------------+-------+-------+-------------------+----------------+----------+------------+----+-----+
|             0|      2|    656|               null|               0|       0.0|         0.0|2013|    0|
|             0|      2|    784|               null|               0|       0.0|         0.0|2013|    0|
|             0|      2|   1409|2013-01-17 00:00:00|              19|    1398.5|         1.0|2013|    0|
|             0|      2|   1988|               null|               0|       0.0|         0.0|2013|    0|
|             0|      2|   2191|               null|               0|       0.0|         0.0|2013|    0|
|             0|      2|   2283|               null|               0|       0.0|         0.0|2013|    0|
|             0|      2|   2856|               null|   

In [22]:
from pyspark.sql import Window

In [23]:
train_monthly = train_monthly.filter(train_monthly.item_cnt_day>=0)

In [24]:
train_monthly = train_monthly.filter(train_monthly.item_cnt_day<=20)
train_monthly = train_monthly.filter(train_monthly.item_price< 400000)

In [25]:
train_monthly.show()

+--------------+-------+-------+-------------------+----------------+----------+------------+----+-----+
|date_block_num|shop_id|item_id|               date|item_category_id|item_price|item_cnt_day|year|month|
+--------------+-------+-------+-------------------+----------------+----------+------------+----+-----+
|             0|      2|    656|               null|               0|       0.0|         0.0|2013|    0|
|             0|      2|    784|               null|               0|       0.0|         0.0|2013|    0|
|             0|      2|   1409|2013-01-17 00:00:00|              19|    1398.5|         1.0|2013|    0|
|             0|      2|   1988|               null|               0|       0.0|         0.0|2013|    0|
|             0|      2|   2191|               null|               0|       0.0|         0.0|2013|    0|
|             0|      2|   2283|               null|               0|       0.0|         0.0|2013|    0|
|             0|      2|   2856|               null|   

In [26]:
train_monthly = train_monthly.withColumn("item_cnt_month", F.lag("item_cnt_day",1).over(Window.partitionBy("shop_id", "item_id").orderBy("date_block_num")))

In [27]:
train_monthly = train_monthly.withColumn('item_cnt_mean', F.avg('item_cnt_day').over(Window.partitionBy("shop_id", "item_category_id", "item_id").orderBy("date_block_num").rangeBetween(-2,0)))


In [28]:
train_monthly = train_monthly.withColumn('item_cnt_std', F.stddev('item_cnt_day').over(Window.partitionBy("shop_id", "item_category_id", "item_id").orderBy("date_block_num").rangeBetween(-2,0)))

In [29]:
train_monthly = train_monthly.withColumn("item_cnt_shifted1", F.lag("item_cnt_day",1).over(Window.partitionBy("shop_id", "item_category_id", "item_id").orderBy("date_block_num")))


In [30]:
train_monthly = train_monthly.withColumn("item_cnt_shifted2", F.lag("item_cnt_day",2).over(Window.partitionBy("shop_id", "item_category_id", "item_id").orderBy("date_block_num")))


In [31]:
train_monthly = train_monthly.withColumn("item_cnt_shifted3", F.lag("item_cnt_day",3).over(Window.partitionBy("shop_id", "item_category_id", "item_id").orderBy("date_block_num")))


In [32]:
train_monthly = train_monthly.fillna(0)

In [33]:
train_monthly.show(20)

+--------------+-------+-------+----+----------------+----------+------------+----+-----+--------------+-------------+------------+-----------------+-----------------+-----------------+
|date_block_num|shop_id|item_id|date|item_category_id|item_price|item_cnt_day|year|month|item_cnt_month|item_cnt_mean|item_cnt_std|item_cnt_shifted1|item_cnt_shifted2|item_cnt_shifted3|
+--------------+-------+-------+----+----------------+----------+------------+----+-----+--------------+-------------+------------+-----------------+-----------------+-----------------+
|             0|      2|   3239|null|               0|       0.0|         0.0|2013|    0|           0.0|          0.0|         0.0|              0.0|              0.0|              0.0|
|             1|      2|   3239|null|               0|       0.0|         0.0|2013|    1|           0.0|          0.0|         0.0|              0.0|              0.0|              0.0|
|             2|      2|   3239|null|               0|       0.0|     

# Spilt Train datasets  validation datasets and test datasets

In [34]:
train_set = train_monthly.filter(train_monthly['date_block_num'] >= 3)
train_set=train_monthly.filter(train_monthly['date_block_num']<=28)


In [35]:
validation_set = train_monthly.filter(train_monthly['date_block_num'] >= 28)
validation_set = train_monthly.filter(train_monthly['date_block_num'] < 33)

In [36]:
test_set = train_monthly.filter(train_monthly['date_block_num'] == 33)

In [37]:
train_set.dropna(subset=['item_cnt_month'])

DataFrame[date_block_num: bigint, shop_id: bigint, item_id: bigint, date: string, item_category_id: bigint, item_price: float, item_cnt_day: float, year: int, month: int, item_cnt_month: float, item_cnt_mean: double, item_cnt_std: double, item_cnt_shifted1: float, item_cnt_shifted2: float, item_cnt_shifted3: float]

In [38]:
validation_set.dropna(subset=['item_cnt_month'])

DataFrame[date_block_num: bigint, shop_id: bigint, item_id: bigint, date: string, item_category_id: bigint, item_price: float, item_cnt_day: float, year: int, month: int, item_cnt_month: float, item_cnt_mean: double, item_cnt_std: double, item_cnt_shifted1: float, item_cnt_shifted2: float, item_cnt_shifted3: float]

In [39]:
train_set.dropna()
validation_set.dropna()

DataFrame[date_block_num: bigint, shop_id: bigint, item_id: bigint, date: string, item_category_id: bigint, item_price: float, item_cnt_day: float, year: int, month: int, item_cnt_month: float, item_cnt_mean: double, item_cnt_std: double, item_cnt_shifted1: float, item_cnt_shifted2: float, item_cnt_shifted3: float]

In [40]:
Total_count=train_monthly.count()
train_count=train_set.count()
val_count=validation_set.count()
test_count=test_set.count()

In [41]:
print('Train set records:', train_count)
print('Validation set records:', val_count)
print('Test set records:', test_count)
print('Train set records: %s (%.f%% of complete data)' % (train_count, ((train_count*100/Total_count))))
print('Validation set records: %s (%.f%% of complete data)' % (val_count, ((val_count*100/Total_count))))
print('Test set records: %s (%.f%% of complete data)' % (test_count, ((test_count*100/Total_count))))

('Train set records:', 6262743)
('Validation set records:', 7134861)
('Test set records:', 218763)
Train set records: 6262743 (85% of complete data)
Validation set records: 7134861 (97% of complete data)
Test set records: 218763 (2% of complete data)

In [42]:
train_set = train_set.withColumn('gp_shop_mean', F.avg('item_cnt_month').over(Window.partitionBy("shop_id")))
train_set = train_set.withColumn('gp_item_mean', F.avg('item_cnt_month').over(Window.partitionBy("item_id")))
validation_set = validation_set.withColumn('gp_shop_mean', F.avg('item_cnt_month').over(Window.partitionBy("shop_id")))
validation_set = validation_set.withColumn('gp_item_mean', F.avg('item_cnt_month').over(Window.partitionBy("item_id")))
test_set = test_set.withColumn('gp_shop_mean', F.avg('item_cnt_month').over(Window.partitionBy("shop_id")))
test_set = test_set.withColumn('gp_item_mean', F.avg('item_cnt_month').over(Window.partitionBy("item_id")))
test_set.show(10)


+--------------+-------+-------+-------------------+----------------+----------+------------+----+-----+--------------+-------------+------------+-----------------+-----------------+-----------------+-------------------+------------+
|date_block_num|shop_id|item_id|               date|item_category_id|item_price|item_cnt_day|year|month|item_cnt_month|item_cnt_mean|item_cnt_std|item_cnt_shifted1|item_cnt_shifted2|item_cnt_shifted3|       gp_shop_mean|gp_item_mean|
+--------------+-------+-------+-------------------+----------------+----------+------------+----+-----+--------------+-------------+------------+-----------------+-----------------+-----------------+-------------------+------------+
|            33|     26|    474|               null|               0|       0.0|         0.0|2015|    9|           0.0|          0.0|         0.0|              0.0|              0.0|              0.0|0.24257713953037066|         0.0|
|            33|     22|    474|2015-10-04 00:00:00|            

In [43]:
train = train_set.drop('date_block_num')
validation = validation_set.drop('date_block_num')
test = test_set.drop('date_block_num', 'item_cnt_month')

In [44]:
test = sale_test.join(test, on=['shop_id', 'item_id'], how="full")

In [45]:
test = test.fillna(0)

In [46]:
test.drop('ID')

DataFrame[shop_id: bigint, item_id: bigint, date: string, item_category_id: bigint, item_price: float, item_cnt_day: float, year: int, month: int, item_cnt_mean: double, item_cnt_std: double, item_cnt_shifted1: float, item_cnt_shifted2: float, item_cnt_shifted3: float, gp_shop_mean: double, gp_item_mean: double]

In [47]:
train.drop('item_category_id')
validation.drop('item_category_id')
test.drop('item_category_id')

DataFrame[shop_id: bigint, item_id: bigint, ID: bigint, date: string, item_price: float, item_cnt_day: float, year: int, month: int, item_cnt_mean: double, item_cnt_std: double, item_cnt_shifted1: float, item_cnt_shifted2: float, item_cnt_shifted3: float, gp_shop_mean: double, gp_item_mean: double]

In [48]:
rf_features = ['shop_id', 
               'item_id', 
               'item_cnt_day',  
               'year', 
               'item_cnt_mean', 
               'item_cnt_std', 
               'item_cnt_shifted1', 
               'item_cnt_month']

In [49]:
rf_train = train[rf_features]
rf_val = validation[rf_features]

In [50]:
test_features = ['shop_id',
                 'item_id', 
                 'item_cnt_day', 
                 'year', 
                 'item_cnt_mean',
                 'item_cnt_std', 
                 'item_cnt_shifted1']

# Model Creation And Training


In [51]:
from pyspark.ml.regression import RandomForestRegressor
from pyspark.ml.feature import VectorAssembler

In [52]:
assembler=VectorAssembler(inputCols=['shop_id','item_id','year', 
                                     'item_cnt_mean', 
                                     'item_cnt_std',
                                     'item_cnt_shifted1'], outputCol="features")

In [53]:
transformed = assembler.transform(rf_train)

In [54]:
rf = RandomForestRegressor(featuresCol="features", labelCol="item_cnt_month")

In [55]:
rf_test = test[test_features]
rf_test.show(10)

+-------+-------+------------+----+-------------+------------------+-----------------+
|shop_id|item_id|item_cnt_day|year|item_cnt_mean|      item_cnt_std|item_cnt_shifted1|
+-------+-------+------------+----+-------------+------------------+-----------------+
|      2|   3239|         0.0|2015|          0.0|               0.0|              0.0|
|      2|   3442|         1.0|2015|          1.2|0.4472135954999579|              2.0|
|      2|   3442|         1.0|2015|          1.2|0.4472135954999579|              1.0|
|      2|   3479|         1.0|2015|          1.0|               0.0|              1.0|
|      2|   5997|         0.0|2015|          0.0|               0.0|              0.0|
|      2|   7879|         0.0|2015|          0.0|               0.0|              0.0|
|      2|   7893|         0.0|2015|          0.0|               0.0|              0.0|
|      2|   8555|         0.0|2015|          0.0|               0.0|              0.0|
|      2|   8702|         0.0|2015|        

In [56]:
rf_model = rf.fit(transformed)

In [57]:
print("Feature Importances: " + str(rf_model.featureImportances))

Feature Importances: (6,[0,1,2,3,4,5],[0.00083371622699,0.0133687748993,0.00283684925131,0.246169685831,0.217829382093,0.518961591698])

In [58]:
rf_val_pred = rf_model.transform(assembler.transform(rf_val))
predictions = rf_model.transform(assembler.transform(rf_test))

In [59]:
predictions = predictions.join(sale_test, ['item_id', 'shop_id'])
predictions.show()

+-------+-------+------------+----+-------------+------------+-----------------+--------------------+--------------------+------+
|item_id|shop_id|item_cnt_day|year|item_cnt_mean|item_cnt_std|item_cnt_shifted1|            features|          prediction|    ID|
+-------+-------+------------+----+-------------+------------+-----------------+--------------------+--------------------+------+
|     33|     36|         1.0|2015|          1.0|         0.0|              0.0|[36.0,33.0,2015.0...|  0.1173424560874911|165292|
|     51|      6|         0.0|2015|          0.0|         0.0|              0.0|[6.0,51.0,2015.0,...|0.056469838719640966| 11891|
|     51|     18|         0.0|2015|          0.0|         0.0|              0.0|[18.0,51.0,2015.0...|0.056469838719640966| 88391|
|     62|     15|         0.0|2015|          0.0|         0.0|              0.0|[15.0,62.0,2015.0...|0.056469838719640966| 78519|
|     83|     25|         0.0|   0|          0.0|         0.0|              0.0|(6,[0,1],[

In [60]:
rf_train_pred = rf_model.transform(transformed)


In [61]:
from pyspark.ml.evaluation import RegressionEvaluator
evaluator = RegressionEvaluator(labelCol = "item_cnt_month", predictionCol="prediction", metricName="rmse")
rmse = evaluator.evaluate(rf_train_pred)
print("Root Mean Squared Error (RMSE) on training data = %g" % rmse)
rmse_val = evaluator.evaluate(rf_val_pred)
print("Root Mean Squared Error (RMSE) on validation data = %g" % rmse_val)

Root Mean Squared Error (RMSE) on training data = 0.282461
Root Mean Squared Error (RMSE) on validation data = 0.291567

In [62]:
predictions = predictions.orderBy("ID")
predictions.show()

+-------+-------+------------+----+-------------+------------+-----------------+--------------------+--------------------+---+
|item_id|shop_id|item_cnt_day|year|item_cnt_mean|item_cnt_std|item_cnt_shifted1|            features|          prediction| ID|
+-------+-------+------------+----+-------------+------------+-----------------+--------------------+--------------------+---+
|   5037|      5|         0.0|2015|          0.0|         0.0|              0.0|[5.0,5037.0,2015....| 0.06516452139338583|  0|
|   5320|      5|         0.0|   0|          0.0|         0.0|              0.0|(6,[0,1],[5.0,532...|0.032701481563419926|  1|
|   5233|      5|         1.0|2015|          1.0|         0.0|              1.0|[5.0,5233.0,2015....|   0.718610892180956|  2|
|   5232|      5|         0.0|2015|          0.0|         0.0|              0.0|[5.0,5232.0,2015....| 0.06516452139338583|  3|
|   5268|      5|         0.0|   0|          0.0|         0.0|              0.0|(6,[0,1],[5.0,526...|0.03270148

In [63]:
predictedSales = predictions.select(F.col("ID"), F.col("Prediction").alias("item_cnt_month")).orderBy(F.col("ID"))
predictedSales.show()

+---+--------------------+
| ID|      item_cnt_month|
+---+--------------------+
|  0| 0.06516452139338583|
|  1|0.032701481563419926|
|  2|   0.718610892180956|
|  3| 0.06516452139338583|
|  4|0.032701481563419926|
|  5|   0.718610892180956|
|  6|   0.718610892180956|
|  6|   0.718610892180956|
|  7| 0.06516452139338583|
|  8| 0.06516452139338583|
|  9| 0.06516452139338583|
| 10|   0.718610892180956|
| 10|   0.718610892180956|
| 10|   0.718610892180956|
| 11| 0.06516452139338583|
| 12| 0.06516452139338583|
| 13| 0.06516452139338583|
| 14|   0.718610892180956|
| 14|   0.718610892180956|
| 15|   0.718610892180956|
+---+--------------------+
only showing top 20 rows

In [64]:
from pyspark.sql import SQLContext
predictedSales.write.saveAsTable('predictedSales')

In [65]:
predictedSales.printSchema()

root
 |-- ID: long (nullable = true)
 |-- item_cnt_month: double (nullable = false)

In [66]:
predictedSales.select("item_cnt_month").show()

+--------------------+
|      item_cnt_month|
+--------------------+
| 0.06516452139338583|
|0.032701481563419926|
|   0.718610892180956|
| 0.06516452139338583|
|0.032701481563419926|
|   0.718610892180956|
|   0.718610892180956|
|   0.718610892180956|
| 0.06516452139338583|
| 0.06516452139338583|
| 0.06516452139338583|
|   0.718610892180956|
|   0.718610892180956|
|   0.718610892180956|
| 0.06516452139338583|
| 0.06516452139338583|
| 0.06516452139338583|
|   0.718610892180956|
|   0.718610892180956|
|   0.718610892180956|
+--------------------+
only showing top 20 rows

In [67]:
%%sql
SELECT * FROM predictedSales